In [1]:
# Importing libraries
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import tqdm
from tqdm.auto import tqdm;

d:\Cinthia\env_modelos\venv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the manual seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [49]:
# Make a device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The device is: {device}")
n_gpus = torch.cuda.device_count()
print(f"Number of gpus: {n_gpus}")
!nvidia-smi --query-gpu=name --format=csv,noheader

The device is: cuda
Number of gpus: 1
NVIDIA GeForce RTX 2060


## 1. Some Blocks


In [50]:
class ConvBlock (nn.Module):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 **kwargs): # this means that the key arguments are arbitrary
        super().__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels,
                              **kwargs,
                              device=device)
        self.batchnorm = nn.BatchNorm2d(num_features=out_channels) # to improve performance
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

In [51]:
class  Inception2 (nn.Module):
    def __init__(self,
                 
                 in_channels: int,
                 red_3x3: int,
                 out_3x3: int):
        
        super().__init__()

        self.branch2 = nn.Sequential(
            ConvBlock(in_channels=in_channels,
                      out_channels=red_3x3,
                      kernel_size=1),
            ConvBlock(in_channels=red_3x3,
                      out_channels=out_3x3,
                      kernel_size=3,
                      padding=1) # ojo (btw no ponemos el S bc por defecto es 1)
        )

    def forward(self,x):
        # N x filters x 28 x 28 → 0th x 1st x 2nd x 3rd dimension (we use 1)
        return self.branch2(x)

In [52]:
class L2NormLayer (nn.Module):
    def __init__(self,dim=1):
        super().__init__()
        self.dim=dim

    def forward (self,x):
        return nn.functional.normalize(x,p=2,dim=self.dim)

In [139]:
from typing import Literal

class Inception3c4e (nn.Module):
    def __init__(self,
                 
                 in_channels: int,
                 
                 red_3x3: int,
                 out_3x3: int,
                 
                 red_5x5: int,
                 out_5x5: int): ## change later
        
        super().__init__()
        
        ## Branch 2
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels=in_channels,
                      out_channels=red_3x3,
                      kernel_size=1),
            ConvBlock(in_channels=red_3x3,
                      out_channels=out_3x3,
                      kernel_size=3,
                      padding=1,
                      stride=2) # ojo (btw no ponemos el S bc por defecto es 1)
        )
        
        ## Branch 3
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels=in_channels,
                      out_channels=red_5x5,
                      kernel_size=1),
            ConvBlock(in_channels=red_5x5,
                      out_channels=out_5x5,
                      kernel_size=5,
                      padding=2,
                      stride=2) # ojo (btw no ponemos el S bc por defecto es 1)
        )

        ## Branch 4
        self.branch4 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    def forward(self,x):
        # N x filters x 28 x 28 → 0th x 1st x 2nd x 3rd dimension (we use 1)
        return torch.cat([self.branch2(x),self.branch3(x),self.branch4(x)],1)

In [140]:
from typing import Literal

class InceptionBlock (nn.Module):
    def __init__(self,
                 
                 in_channels: int,
                 out_1x1: int,

                 red_3x3: int,
                 out_3x3: int,

                 red_5x5: int,
                 out_5x5: int,

                 out_1x1pool: int,
                 
                 pool_type: Literal['l2', 'max']):
        
        super().__init__()

        ## Branch 1
        self.branch1 = ConvBlock(in_channels=in_channels,
                                 out_channels=out_1x1,
                                 kernel_size=1)
        
        ## Branch 2
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels=in_channels,
                      out_channels=red_3x3,
                      kernel_size=1),
            ConvBlock(in_channels=red_3x3,
                      out_channels=out_3x3,
                      kernel_size=3,
                      padding=1) # ojo (btw no ponemos el S bc por defecto es 1)
        )
        
        ## Branch 3
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels=in_channels,
                      out_channels=red_5x5,
                      kernel_size=1),
            ConvBlock(in_channels=red_5x5,
                      out_channels=out_5x5,
                      kernel_size=5,
                      padding=2) # ojo (btw no ponemos el S bc por defecto es 1)
        )

        ## Branch 4
        if pool_type == 'max':
            self.branch4 = nn.Sequential(
                nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                ConvBlock(in_channels=in_channels,
                          out_channels=out_1x1pool,
                          kernel_size=1)
            )
        elif pool_type == 'l2':
            self.branch4 = nn.Sequential(
                L2NormLayer(),
                ConvBlock(in_channels=in_channels,
                          out_channels=out_1x1pool,
                          kernel_size=1)
            )

    def forward(self,x):
        # N x filters x 28 x 28 → 0th x 1st x 2nd x 3rd dimension (we use 1)
        output = []

        if self.branch1 is not None:
            output.append(self.branch1(x))
        output.append(self.branch2(x))
        output.append(self.branch3(x))
        output.append(self.branch4(x))

        return torch.cat(output,1)

## 3. The principal functions

In [152]:
class NN2 (nn.Module):
    def __init__(self,
                 in_channels = 3,
                 use_auxiliary = True):
        super().__init__()

        self.conv1 = ConvBlock(in_channels=in_channels,
                               out_channels=64,
                               kernel_size=7,
                               stride=2,
                               padding=3)
        
        # In this order: 'in_channels',red_3x3,out_3x3
        self.inception2 = Inception2(64,64,192)

        # In this order: in_channels,red_3x3,out_3x3,red_5x5,out_5x5
        self.inception3c = Inception3c4e(320,128,256,32, 64)
        self.inception4e = Inception3c4e(640,160,256,64,128)

        # In this order: in_channels,out_1x1,red_3x3,out_3x3,red_5x5,out_5x5,out_1x1pool
        self.inception3a = InceptionBlock( 192, 64, 96,128,16, 32, 32, pool_type='max')
        self.inception3b = InceptionBlock( 256, 64, 96,128,32, 64, 64, pool_type='l2')

        self.inception4a = InceptionBlock( 640,256, 96,192,32, 64,128, pool_type='l2')
        self.inception4b = InceptionBlock( 640,224,112,224,32, 64,128, pool_type='l2')
        self.inception4c = InceptionBlock( 640,192,128,256,32, 64,128, pool_type='l2')
        self.inception4d = InceptionBlock( 640,160,144,288,32, 64,128, pool_type='l2')
        
        self.inception5a = InceptionBlock(1024,384,192,384,48,128,128, pool_type='l2')
        self.inception5b = InceptionBlock(1024,384,192,384,48,128,128, pool_type='max')
        
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.avgpool = nn.AvgPool2d(kernel_size=7) # stride? padding?
        
        self.FC = nn.Linear(1024,128)
        self.norm = nn.BatchNorm2d(num_features=64)

    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.norm(x)
        
        x = self.inception2(x)
        print(f"Inception 2: {x.shape}")

        x = self.maxpool(x)

        x = self.inception3a(x)
        print(f"Inception 3a: {x.shape}")
        x = self.inception3b(x)
        print(f"Inception 3b: {x.shape}")
        x = self.inception3c(x)
        print(f"Inception 3c: {x.shape}")

        x = self.inception4a(x)
        print(f"Inception 4a: {x.shape}")
        x = self.inception4b(x)
        print(f"Inception 4b: {x.shape}")
        x = self.inception4c(x)
        print(f"Inception 4c: {x.shape}")
        x = self.inception4d(x)
        print(f"Inception 4d: {x.shape}")
        x = self.inception4e(x)
        print(f"Inception 4e: {x.shape}")

        x = self.inception5a(x)
        print(f"Inception 5a: {x.shape}")
        x = self.inception5b(x)
        print(f"Inception 5b: {x.shape}")

        x = self.avgpool(x)
        print(f"Avg pool: {x.shape}")
        x = x.view(x.shape[0],-1)
        print(f"View: {x.shape}")
        x = self.FC(x)
        print(f"FC: {x.shape}")
        x = nn.functional.normalize(x,p=2,dim=1)
        print(f"Normalization: {x.shape}")

        return x

## 4. Activate the model

In [153]:
model_1 = NN2().to(device)
print(f"Are the models in the 'cuda' device? {next(model_1.parameters()).is_cuda}")

Are the models in the 'cuda' device? True


## 5. Verify the dimensions

In [154]:
another_random_tensor = torch.randn(12,3,224,224).to(device)
model_1(another_random_tensor)

Inception 2: torch.Size([12, 192, 56, 56])
Inception 3a: torch.Size([12, 256, 28, 28])
Inception 3b: torch.Size([12, 320, 28, 28])
Inception 3c: torch.Size([12, 640, 14, 14])
Inception 4a: torch.Size([12, 640, 14, 14])
Inception 4b: torch.Size([12, 640, 14, 14])
Inception 4c: torch.Size([12, 640, 14, 14])
Inception 4d: torch.Size([12, 640, 14, 14])
Inception 4e: torch.Size([12, 1024, 7, 7])
Inception 5a: torch.Size([12, 1024, 7, 7])
Inception 5b: torch.Size([12, 1024, 7, 7])
Avg pool: torch.Size([12, 1024, 1, 1])
View: torch.Size([12, 1024])
FC: torch.Size([12, 128])
Normalization: torch.Size([12, 128])


tensor([[ 0.0181, -0.0986,  0.0531,  ...,  0.1195,  0.0430, -0.0647],
        [ 0.0320, -0.1091,  0.0205,  ...,  0.1246,  0.0797, -0.0758],
        [ 0.0767, -0.0716,  0.0327,  ...,  0.0721,  0.0551, -0.1118],
        ...,
        [ 0.0154, -0.1524, -0.0127,  ...,  0.0914,  0.0738, -0.0688],
        [ 0.0432, -0.1119,  0.0312,  ...,  0.0995,  0.0507, -0.0395],
        [ 0.0195, -0.1390,  0.0346,  ...,  0.1088,  0.0550, -0.0210]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
random_tensor = torch.randn(51,3,224,224).to(device)

from torchinfo import summary

summary(model_1,random_tensor.shape)

Layer (type:depth-idx)                   Output Shape              Param #
NN2                                      [51, 128]                 --
├─ConvBlock: 1-1                         [51, 64, 112, 112]        --
│    └─Conv2d: 2-1                       [51, 64, 112, 112]        9,472
│    └─BatchNorm2d: 2-2                  [51, 64, 112, 112]        128
│    └─ReLU: 2-3                         [51, 64, 112, 112]        --
├─MaxPool2d: 1-2                         [51, 64, 56, 56]          --
├─BatchNorm2d: 1-3                       [51, 64, 56, 56]          128
├─Inception2: 1-4                        [51, 192, 56, 56]         --
│    └─Sequential: 2-4                   [51, 192, 56, 56]         --
│    │    └─ConvBlock: 3-1               [51, 64, 56, 56]          4,288
│    │    └─ConvBlock: 3-2               [51, 192, 56, 56]         111,168
├─MaxPool2d: 1-5                         [51, 192, 28, 28]         --
├─InceptionBlock: 1-6                    [51, 256, 28, 28]         --
│ 